# Merge Sort
---
With merge sort, we will again try to solve our same problem, but trying to do so in a better (faster) way. So far from the methods we have learned, if we were given a list of number that we had no information about, our best bet would be to use selection sort. Let's first understand how merge sort works and decide if it can compete with selection sort, or if it should only be used with specific cases (i.e. bucket sort). 

### Merge - the idea

Take two lists that are each in sorted order. We can start by keeping track of the smallest (leftmost) item in each list. We compare the items and put the smaller in our new combined list. Move to the next smallest item and start the comparison process over again until the complete full list is filled.

How and why do we start with two lists to make 1?

Let's think backwards. If we break up our list into many lists all of size 1, they are all in sorted order. Remember that any list of size 1 is always in sorted order. One by one we combine our lists together with our process merge. 

How does this process improve on the methods we have already seen?

Merge sort requires ***no swaps*** or ***rearranging*** of items. We will simply move the items into a new list, placing them in order as we go.

The remaining question is: How many marge steps are required for a list of N items?

<img src="https://slidetodoc.com/presentation_image_h/11db954aaa85205f333354e3f428e601/image-19.jpg" />

In practice there are two versions of implementing merge sort. We will focus on the iterative version instead of the recursive one. Don't worry right now if those are unfamiliar words, they will make sense in the near future!

<img src="https://upload.wikimedia.org/wikipedia/commons/c/cc/Merge-sort-example-300px.gif" width="400">

### Steps

1. We first start by processing lists of size 1, 2 at a time to merge together. 
1. Complete the merge step between a pair of lists for all pairs in the list. 
1. If there is an extra item (for an odd sized list), move that item up to the next round. 
1. Double the size of the intervals merged and start again. 

First let's practice merging two sorted lists into one so we fully understand how the merge step works. 

In [55]:
# take in two lists, return one merged list
def merge(a, b):
    merged = []
    a_pos, b_pos = 0, 0 # since lists are sorted, start at each lists smallest value, at index 0
    while a_pos < len(a) and b_pos < len(b): # compare smallest in a to smallest in b, so long as neither are finished
        if a[a_pos] < b[b_pos]:
            merged.append(a[a_pos]) # place the smaller item between a and b in the list
            a_pos += 1 # move to the next index if the value was used
        else:
            merged.append(b[b_pos])
            b_pos += 1
    while a_pos < len(a): # if list b finished, but list a has more items left, empty list a into the merged list
        merged.append(a[a_pos])
        a_pos += 1
    while b_pos < len(b): # same thing if it is the case for b
        merged.append(b[b_pos])
        b_pos += 1
    return merged # send back the merged list


test1 = [1, 4, 7, 9]
test2 = [2, 3, 8]
print(merge(test1, test2))

[1, 2, 3, 4, 7, 8, 9]


This is how the merge step works in practice, but remember we are starting out with a full list and we need to break it down into smaller lists that are the length of the interval (or shorter for extra bits). How can we determine the start and end position for each of the lists we will pass to our merge function?

We need: 
1. list a start position
1. list a end position
1. list b start position
1. list b end position

At the beginning our start for a will always begin at index 0. 

The ending point on a is dependant on the current segment length. Therefore we can say `a_end = start + current_segment_length`. 
Example: if segment length is currently 1, and we are starting from the beginning, our start for a will be index 0, our end for a will be index 1. Essentially out list will stop when it hits the ending position, not include the value at the ending position. 

The start of b will begin at the end of a. 

The end of b will be at `start + current_segment_length`. 

There is one last thing we need to be careful of that we did not include in our intervals. How do we make sure that a_end, b_start and b_end do not go out of bounds?

In order to fix this, we can update our instructions on those three positions, instead of always allowing our ends to be at `pos + current_segment_length` we can instead take `min(len(list), current_segment_length)`. That way if our list goes too far, we can catch it and set the end to the end of our list. 

At the end of each iteration, our segment size will double. We will loop as long as our segment is less than the length of our list. Let's put this all together. 


In [75]:
import random
test = [random.randint(0, 100) for i in range(11)] # generate a random list of 11 numbers (we will learn this syntax soon!)
print('starting list is:', test)

def merge_sort(lst):
    n = len(lst)
    segment_size = 1 # always start with the smallest possible sorted lists, lists of size 1
    while segment_size < n:
        start_pos = 0
        merged_itms = [] # here is where we will put our merged items. we will replace our list with our merged list at the end of each loop
        while start_pos < n:
            a_pos = start_pos # initialize our start and end points for our two lists
            a_end = min(a_pos + segment_size, n)
            b_pos = a_end
            b_end = min(b_pos + segment_size, n)
            c = merge(lst[a_pos:a_end], lst[b_pos:b_end]) # merge the two lists to get a new list of size len(a) + len(b)
            merged_itms.extend(c) # add our merged list to what we have so far
            start_pos += 2 * segment_size # move our starting point to the next interval
        segment_size *= 2 # once we loop through and merge the entire list, double our segment size for next round
        lst = merged_itms # change the list we have to the merged one we just created
    return lst

print('sorted list is:', merge_sort(test))

starting list is: [1, 9, 9, 99, 30, 88, 13, 4, 62, 18, 3]
sorted list is: [1, 3, 4, 9, 9, 13, 18, 30, 62, 88, 99]


### Why is merge sort better than what we have seen so far?

The reason merge sort is such a beautiful solution to sorting lies in the fact that we do not need to complete the merge step many times! Let's see how many times we need to merge to understand the beauty of this solution. 

Let's create a much larger list, and add a variable to our merge sort code to keep track of the number of merges that happen. 

In [70]:
import random
test2 = [random.randint(0, 100) for i in range(65536)]

def merge_sort_counter(lst):
    n = len(lst)
    segment_size = 1
    num_merges = 0 # count the number of merge itervals we try
    while segment_size < n:
        start_pos = 0
        merged_itms = []
        while start_pos < n:
            a_pos = start_pos
            a_end = min(a_pos + segment_size, n)
            b_pos = a_end
            b_end = min(b_pos + segment_size, n)
            c = merge(lst[a_pos:a_end], lst[b_pos:b_end])
            merged_itms.extend(c)
            start_pos += 2 * segment_size 
        segment_size *= 2
        lst = merged_itms
        num_merges += 1 # loop is finished so increase the count
    return num_merges

print('number of items in list:', len(test2))
print('number of merges:', merge_sort_counter(test2))

number of items in list: 65536
number of merges: 16


Why 16? Each time we double our segment size so we are looking at segment sizes of: 

`1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768` and when it hits `65536` it is finished. 

We can also view this as $2^{16} = 65536$ 

In mathematics we have a way of reversing this process. If we start with a number and ask ourself how many times did we double before reaching our number, we can do so taking $log_2$ of the number. 

We can also say $log_2(65536) = 16$

Why is this important? Let's compare our bubble sort against our merge sort. Take our list of 65536 items. 

In bubble sort we pick a value, and compare it against all the rest to bubble it to the top. Thefore in this case, our outer loop would loop 65536 times and our inner loop must visit each of the items in our list. In total we would visit $65536 * 65536 = 4294967296$ items from our list. 

In merge sort our outer loop will iterate $log_2(65536) = 16$ times while our inner loop will visit each of our 65536 items. In total we will visit $16 * 65536 = 1048576$ total items in our list. 

In total we saved $4294967296 - 1048576 = 4293918720$ total item visits. This is a huge amount!!

Now let's see the power of merge sort in action by comparing it with our bucket sort in a speed test. 

In [73]:
import math
import datetime
import random

# make our random lists for testing
bucket_test_lst = []
merge_test_lst = []
for i in range(10000):
    num = random.randint(0, 1000)
    bucket_test_lst.append(num)
    merge_test_lst.append(num)

# set up buckets for bucket sort
b = math.floor(math.sqrt(len(bucket_test_lst))) + 1
bucket_lst = []
for i in range(b):
    bucket_lst.append([])

# code used for our bucket sort
def insertion(b):  
    itm_to_insert = bucket_lst[b][-1] 
    curr_pos = len(bucket_lst[b]) - 1 
    while curr_pos > 0 and itm_to_insert < bucket_lst[b][curr_pos - 1]: 
        bucket_lst[b][curr_pos] = bucket_lst[b][curr_pos - 1]
        curr_pos -= 1
    bucket_lst[b][curr_pos] = itm_to_insert

def bucket(l):
    for itm in l: 
        position = math.floor(itm/b) 
        bucket_lst[position].append(itm)
        if len(bucket_lst[position]) > 1:
            insertion(position)

# code for merge sort in above cells


# testing
start = datetime.datetime.now()
bucket(bucket_test_lst)
total_time = datetime.datetime.now() - start
print('the total time for average case bucket sort is:', total_time)

start = datetime.datetime.now()
merge_sort(merge_test_lst)
total_time = datetime.datetime.now() - start
print('the total time for average case merge sort is:', total_time)

the total time for average case bucket sort is: 0:00:00.811093
the total time for average case merge sort is: 0:00:00.068006


<div class="alert alert-block alert-success"><b>Average case sorting winner:</b> Merge sort!</div>

### Which sorting method does Python use when we call the built in method sort()?

From all of our sorting method testing and understanding, we have learned that for average cases, merge sort is the best sorting method we can use. However, under some special cases, insertion sort is even faster than merge sort. 

Python's approach to sorting: TimSort

TimSort is a hybrid of merge sort and insertion sort. In the best case it is as fast as best case scenereo insertion sort, and at its worst it is as slow as worst case merge sort. 

let's do a final test to see how our merge sort code compares against Pythons TimSort.

In [74]:
merge_test_lst = []
timsort_test_lst = []
for i in range(10000):
    num = random.randint(0, 1000)
    merge_test_lst.append(num)
    timsort_test_lst.append(num)


start = datetime.datetime.now()
merge_sort(merge_test_lst)
total_time = datetime.datetime.now() - start
print('the total time for average case merge sort is:', total_time)

start = datetime.datetime.now()
timsort_test_lst.sort()
total_time = datetime.datetime.now() - start
print('the total time for average case timsort is:', total_time)

the total time for average case merge sort is: 0:00:00.067599
the total time for average case timsort is: 0:00:00.001098


### Summary

If Python's sort() works better than any of the sorting methods we have learned, why do we have methods like bubble and selection sort?

The algorithms we learn today are only the best that we know so far. There is a possibility that there are better solutions that exist, but have not yet been discovered. In order to find better solutions to problems we have already solved, we need to study the flaws in our algorithms and create ways to improve on them. 